In [28]:
import cv2
import pandas as pd
import os
import random
import sys

sys.path.append('../')

from utils.io import load_yaml

servers = load_yaml('config/servers.yml')
server = 'local'

In [42]:
prefix = servers[server]['database']['location']
name = 'camera_motion_separated'
prefix = os.path.join(prefix, name)
df_log = pd.read_pickle(os.path.join(prefix, 'log.pkl'))

df_log[df_log['group'] == 'without_camera_motion']['video'].to_csv('without_motion.csv', index=False)
df_log[df_log['group'] == 'with_camera_motion']['video'].to_csv('with_motion.csv', index=False)

In [32]:
df_without_camera_motion_split = pd.read_csv('config/endoscopic_datasets_data_splits_without_camera_motion.csv')
df_without_camera_motion_split

,camera_motion,video,group,snippet,length,sum,target test length,test set,tmp,actual test length,total target test,actual total test
0,False,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,without_camera_motion/vid_0.mp4,1520,26340.0,5268,True,1520,3580.0,"39568,2",38407.0
1,False,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,without_camera_motion/vid_1.mp4,560,NaN,NaN,True,560,NaN,NaN,NaN
2,False,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,without_camera_motion/vid_2.mp4,1060,NaN,NaN,True,1060,NaN,NaN,NaN
3,False,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,without_camera_motion/vid_3.mp4,440,NaN,NaN,True,440,NaN,NaN,NaN
4,False,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,without_camera_motion/vid_4.mp4,8260,NaN,NaN,False,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
444,False,left.avi,Mountney,without_camera_motion/vid_444.avi,2188,NaN,NaN,False,0,NaN,NaN,NaN
445,False,left.avi,Mountney,without_camera_motion/vid_445.avi,2698,NaN,NaN,True,2698,NaN,NaN,NaN
446,False,left.avi,Mountney,without_camera_motion/vid_446.avi,1109,NaN,NaN,False,0,NaN,NaN,NaN
447,False,left.avi,Giannarou,without_camera_motion/vid_447.avi,1409,7943.0,"1588,6",True,1409,1409.0,NaN,NaN


In [33]:
df_with_camera_motion_split = pd.read_csv('config/endoscopic_datasets_data_splits_with_camera_motion.csv')
df_with_camera_motion_split

,camera_motion,video,group,snippet,length,sum,target test length,test set,tmp,actual test length,total target test,actual total test
0,True,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,with_camera_motion/vid_0.mp4,60,680.0,136.0,True,60,120.0,2052.0,2180.0
1,True,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,with_camera_motion/vid_1.mp4,200,NaN,NaN,False,0,NaN,NaN,NaN
2,True,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,with_camera_motion/vid_2.mp4,120,NaN,NaN,False,0,NaN,NaN,NaN
3,True,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,with_camera_motion/vid_3.mp4,120,NaN,NaN,False,0,NaN,NaN,NaN
4,True,test/test_video_0002.mp4,SurgVisDom_test_no_indicators,with_camera_motion/vid_4.mp4,60,NaN,NaN,True,60,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
185,True,left.avi,Mountney,with_camera_motion/vid_185.avi,120,NaN,NaN,False,0,NaN,NaN,NaN
186,True,left.avi,Mountney,with_camera_motion/vid_186.avi,150,NaN,NaN,False,0,NaN,NaN,NaN
187,True,left.avi,Mountney,with_camera_motion/vid_187.avi,120,NaN,NaN,True,120,NaN,NaN,NaN
188,True,left.avi,Mountney,with_camera_motion/vid_188.avi,150,NaN,NaN,False,0,NaN,NaN,NaN


In [38]:
# split data
out_prefix = prefix

# without motion
for idx, row in df_without_camera_motion_split.iterrows():
    if row['test set'] == True:
        # move to test
        snippet = row.snippet.replace('without_camera_motion/', '')
        video = 'without_camera_motion/test/{}'.format(snippet)
        os.rename(os.path.join(out_prefix, row.snippet), os.path.join(out_prefix, video))

        # find in df_log, rename video -> test folder
        idx = df_log.index[df_log['video'] == row.snippet].tolist()[0]
        df_log['video'][idx] = video
    else:
        snippet = row.snippet.replace('without_camera_motion/', '')
        video = 'without_camera_motion/train/{}'.format(snippet)
        os.rename(os.path.join(out_prefix, row.snippet), os.path.join(out_prefix, video))
        
        # find in df_log, rename video -> test folder
        idx = df_log.index[df_log['video'] == row.snippet].tolist()[0]
        df_log['video'][idx] = video

# with motion
for idx, row in df_with_camera_motion_split.iterrows():
    if row['test set'] == True:
        # move to test
        snippet = row.snippet.replace('with_camera_motion/', '')
        video = 'with_camera_motion/test/{}'.format(snippet)
        os.rename(os.path.join(out_prefix, row.snippet), os.path.join(out_prefix, video))

        # find in df_log, rename video -> test folder
        idx = df_log.index[df_log['video'] == row.snippet].tolist()[0]
        df_log['video'][idx] = video
    else:
        snippet = row.snippet.replace('with_camera_motion/', '')
        video = 'with_camera_motion/train/{}'.format(snippet)
        os.rename(os.path.join(out_prefix, row.snippet), os.path.join(out_prefix, video))
        
        # find in df_log, rename video -> test folder
        idx = df_log.index[df_log['video'] == row.snippet].tolist()[0]
        df_log['video'][idx] = video

df_log.to_pickle(os.path.join(out_prefix, 'log.pkl'))
df_log.to_csv(os.path.join(out_prefix, 'log.csv'))
# os.rename()

In [16]:
df_log_with_motion = df_log[df_log['group'] == 'with_camera_motion']
df_log_without_motion = df_log[df_log['group'] == 'without_camera_motion']

dfs = {'with_motion': df_log_with_motion, 'without_motion': df_log_without_motion}

for key in dfs:
    total_images = 0
    for idx, row in dfs[key].iterrows():
        vc = cv2.VideoCapture(os.path.join(prefix, row.video))
        total_images += int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

    print(key, total_images)

with_motion 10260
without_motion 197841


In [27]:
# find all snippets that belong to one video

# with motion
df_motion_split_with_motion = pd.DataFrame(columns=['camera_motion', 'video', 'group', 'snippet', 'length'])
df_motion_split_without_motion = pd.DataFrame(columns=['camera_motion', 'video', 'group', 'snippet', 'length'])

print('With motion')
videos = {}
for idx, row in df_log_with_motion.iterrows():
    if row.origin['Video'] not in videos:
        videos[row.origin['Video']] = {}
    vc = cv2.VideoCapture(os.path.join(prefix, row.video))
    videos[row.origin['Video']][row.video] = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

    df_motion_split_with_motion = df_motion_split_with_motion.append({
        'camera_motion': True,
        'video': row.origin['Video'],
        'group': row.origin['Group'],
        'snippet': row.video,
        'length': int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    }, ignore_index=True)

# iterate over all combinations, find best split NOT POSSIBLE, 79! combinations e.g. e115
for key in videos:
    print(key, len(videos[key]))
    # print(random.sample(list(videos[key]), int(0.2*len(videos[key]))))
print('\n')


# without motion
print('Without motion')
videos = {}
for idx, row in df_log_without_motion.iterrows():
    if row.origin['Video'] not in videos:
        videos[row.origin['Video']] = {}
    vc = cv2.VideoCapture(os.path.join(prefix, row.video))
    videos[row.origin['Video']][row.video] = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

    df_motion_split_without_motion = df_motion_split_without_motion.append({
        'camera_motion': False,
        'video': row.origin['Video'],
        'group': row.origin['Group'],
        'snippet': row.video,
        'length': int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    }, ignore_index=True)

# df_motion_split_with_motion.to_csv('with_motion.csv', index=False)
# df_motion_split_without_motion.to_csv('without_motion.csv', index=False)

# iterate over all combinations, find best split NOT POSSIBLE, 79! combinations e.g. e115
for key in videos:
    print(key, len(videos[key]))



With motion
test/test_video_0002.mp4 6
test/test_video_0003.mp4 11
test/test_video_0005.mp4 1
test/test_video_0006.mp4 1
test/test_video_0007.mp4 2
test/test_video_0010.mp4 79
test/test_video_0011.mp4 8
test/test_video_0013.mp4 4
test/test_video_0014.mp4 6
test/test_video_0000.mp4 18
test/test_video_0004.mp4 4
test/test_video_0008.mp4 10
test/test_video_0009.mp4 2
test/test_video_0012.mp4 3
train_1/Porcine/Dissection/DS_0001.mp4 1
train_1/Porcine/Dissection/DS_0002.mp4 3
train_1/Porcine/Dissection/DS_0003.mp4 7
train_1/Porcine/Dissection/DS_0004.mp4 5
train_1/Porcine/Dissection/DS_0005.mp4 1
train_1/Porcine/Dissection/DS_0006.mp4 1
train_1/Porcine/Dissection/DS_0007.mp4 3
train_1/Porcine/Dissection/DS_0008.mp4 1
train_1/Porcine/Needle_Driving/ND_0007.mp4 1
train_1/Porcine/Needle_Driving/ND_0008.mp4 2
train_1/Porcine/Needle_Driving/ND_0009.mp4 2
train_1/Porcine/Needle_Driving/ND_0010.mp4 2
left.avi 6


Without motion
test/test_video_0002.mp4 7
test/test_video_0003.mp4 12
test/test_video

In [35]:
origins = [
    'SurgVisDom_test_no_indicators',
    'SurgVisDom_test_indicators',
    'SurgVisDom_train_indicators',
    'Giannarou',
    'Mountney'
]

for origin in origins:
    total_images = 0
    for idx, row in df_log.iterrows():
        if origin != row['origin']['Group']:
            continue
        else:
            vc = cv2.VideoCapture(os.path.join(prefix, row.video))
            total_images += int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

    print(origin, total_images)

SurgVisDom_test_no_indicators 128280
SurgVisDom_test_indicators 40800
SurgVisDom_train_indicators 16540
Giannarou 8063
Mountney 14418


In [27]:
folders = {
    'SARAS_ESAD': [
        '/media/martin/Samsung_T5/data/endoscopic_data/SARAS_ESAD/train/set1',
        '/media/martin/Samsung_T5/data/endoscopic_data/SARAS_ESAD/train/set2',
        '/media/martin/Samsung_T5/data/endoscopic_data/SARAS_ESAD/val/obj'
    ],
    'KBD': [
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_testing/kidney_dataset_1/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_testing/kidney_dataset_2/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_testing/kidney_dataset_3/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_testing/kidney_dataset_4/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_training/kidney_dataset_1/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_training/kidney_dataset_2/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_training/kidney_dataset_3/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_1_4_training/kidney_dataset_4/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_5/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_6/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_7/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_8/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_9/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_10/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_11/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_12/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_13/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_14/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_testing/kidney_dataset_15/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_5/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_6/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_7/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_8/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_9/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_10/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_11/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_12/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_13/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_14/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_5_15_training/kidney_dataset_15/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_16_20_testing/kidney_dataset_16/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_16_20_testing/kidney_dataset_17/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_16_20_testing/kidney_dataset_18/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_16_20_testing/kidney_dataset_19/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/KBD/kidney_16_20_testing/kidney_dataset_20/left_frames'
    ],
    'RSS': [
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_2018_release_1/miccai_challenge_2018_release_1/seq_1/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_2018_release_1/miccai_challenge_2018_release_1/seq_2/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_2018_release_1/miccai_challenge_2018_release_1/seq_3/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_2018_release_1/miccai_challenge_2018_release_1/seq_4/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_2/seq_5/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_2/seq_6/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_2/seq_7/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_3/seq_9/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_3/seq_10/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_3/seq_11/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_3/seq_12/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_4/seq_13/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_4/seq_14/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_4/seq_15/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RSS/miccai_challenge_release_4/seq_16/left_frames'
    ],
    'RIS': [
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_testing/instrument_dataset_1/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_testing/instrument_dataset_2/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_testing/instrument_dataset_3/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_testing/instrument_dataset_4/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_training/instrument_dataset_1/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_training/instrument_dataset_2/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_training/instrument_dataset_3/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_1_4_training/instrument_dataset_4/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_testing/instrument_dataset_5/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_testing/instrument_dataset_6/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_testing/instrument_dataset_7/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_testing/instrument_dataset_8/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_training/instrument_dataset_5/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_training/instrument_dataset_6/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_training/instrument_dataset_7/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_5_8_training/instrument_dataset_8/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_9_10_testing/instrument_dataset_9/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_9_10_testing/instrument_dataset_10/left_frames',
        '/media/martin/Samsung_T5/data/endoscopic_data/RIS/instrument_dataset_1/left_frames'
    ],
}

postfix = {
    'SARAS_ESAD': '.jpg',
    'KBD': '.png',
    'RSS': '.png',
    'RIS': '.png'
}

for key in folders:
    total_count = 0
    for folder in folders[key]:
        df = scan2df(folder, postfix[key])
        total_count += int(len(df))
    print(key, total_count, 'images')

SARAS_ESAD 47282 images
KBD 3000 images
RSS 2235 images
RIS 3225 images


In [39]:
cholec_df = scan2df('/media/martin/Samsung_T5/data/endoscopic_data/cholec80/videos', '.mp4')

total_frames = 0
for idx, row in cholec_df.iterrows():
    vc = cv2.VideoCapture(os.path.join(row.folder, row.file))
    total_frames += int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

print('cholec80', total_frames, 'frames')

cholec80 4612530 frames


In [59]:
folders = {
    'SurgVisDom': [
        '/media/martin/Samsung_T5/data/endoscopic_data/SurgVisDom/test',
        '/media/martin/Samsung_T5/data/endoscopic_data/SurgVisDom/train_1/Porcine/Dissection',
        '/media/martin/Samsung_T5/data/endoscopic_data/SurgVisDom/train_1/Porcine/Knot_Tying',
        '/media/martin/Samsung_T5/data/endoscopic_data/SurgVisDom/train_1/Porcine/Needle_Driving',
    ],
    'Giannarou': [
        '/media/martin/Samsung_T5/data/endoscopic_data/Giannarou'
    ],
    'Mountney': [
        '/media/martin/Samsung_T5/data/endoscopic_data/Mountney'
    ]
}

postfix = {
    'SurgVisDom': '.mp4',
    'Giannarou': '.avi',
    'Mountney': '.avi'
}

for key in folders:
    for folder in folders[key]:
        df = scan2df(folder, postfix[key])

        for idx, row in df.iterrows():
            vc = cv2.VideoCapture(os.path.join(row.folder, row.file))
            size = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
            test_size = int(0.2*size)
            train_size = size - test_size

            print(row.file, ': test_size', test_size, 'train_size', train_size)

test_video_0015.mp4 : test_size 841 train_size 3367
test_video_0014.mp4 : test_size 5779 train_size 23116
test_video_0013.mp4 : test_size 2809 train_size 11237
test_video_0012.mp4 : test_size 1770 train_size 7084
test_video_0011.mp4 : test_size 932 train_size 3728
test_video_0010.mp4 : test_size 4426 train_size 17704
test_video_0009.mp4 : test_size 1333 train_size 5333
test_video_0008.mp4 : test_size 2102 train_size 8409
test_video_0007.mp4 : test_size 1254 train_size 5018
test_video_0006.mp4 : test_size 2094 train_size 8379
test_video_0005.mp4 : test_size 1251 train_size 5006
test_video_0004.mp4 : test_size 787 train_size 3148
test_video_0003.mp4 : test_size 847 train_size 3391
test_video_0002.mp4 : test_size 5402 train_size 21611
test_video_0001.mp4 : test_size 1074 train_size 4298
test_video_0000.mp4 : test_size 1092 train_size 4371
DS_0008.mp4 : test_size 268 train_size 1072
DS_0007.mp4 : test_size 736 train_size 2944
DS_0006.mp4 : test_size 80 train_size 320
DS_0005.mp4 : test_siz